In [2]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv
import os
from PIL import ImageFont, ImageDraw, Image
import shutil

In [3]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [16]:
input_dir = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_good\\'
output_dir = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_middle\\'
cropped_dir = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_fonts\\'

font_names = []
font_files = [u'simhei.ttf', u'simsun.ttc', u'msjhbd.ttc', u'msjhl.ttc', u'msjh.ttc',
             u'msjhbd.ttc', u'msjhl.ttc', u'msjh.ttc', u'msyhbd.ttc', u'msyhl.ttc', u'msyh.ttc', u'msyhbd.ttc', u'msyhl.ttc',
             u'msyh.ttc', u'mingliu.ttc', u'mingliu.ttc', u'simfang.ttf']
for font_file in font_files:
    font_names.append(font_file[:font_file.find('.')])

image_mode = 'L'

size = 46

print font_names

In [17]:
def prepare_dir(dir_name, subdir_name):
    if os.path.exists(dir_name + subdir_name):
        shutil.rmtree(dir_name + subdir_name)
    
    os.makedirs(dir_name + subdir_name)

In [18]:
def draw_hieroglyph(font, filename, subdir):
    hieroglyph = Image.new(image_mode, (size, int(1.3 * size)), color=255)
    draw = ImageDraw.Draw(hieroglyph)
    draw.text((0, 0), filename[0], font=font, fill=0)
    
    hieroglyph.save(output_dir + subdir + '\\' + filename + '.bmp', 'BMP')

In [19]:
fonts = []

for index in range(len(font_names)):
    prepare_dir(output_dir, font_names[index])
    fonts.append(ImageFont.truetype(font_files[index], size))

for filename in os.listdir(input_dir):
    for index in range(len(font_names)):
        draw_hieroglyph(fonts[index], filename, font_names[index])

Обрежем края на шрифтовых изображениях иероглифов

In [26]:
white_pixel = 255

def has_row_black(image, row):
    width = image.size[0]
    for x in range(0, width):
        if image.getpixel((x, row)) != white_pixel:
            return True
    return False    
    
def has_col_black(image, col):
    height = image.size[1]
    for y in range(0, height):
        if image.getpixel((col, y)) != white_pixel:
            return True
    return False

def crop_image(image):
    width, height = image.size
    
    left = 0
    upper = 0
    right = 0
    lower = 0
    
    for y in range(0, height):
        if has_row_black(image, y):
            upper = y
            break
        
    for x in range(0, width):
        if has_col_black(image, x):
            left = x
            break
    
    for x in range(width - 1, -1, -1):
        if has_col_black(image, x):
            right = x
            break
        
    for y in range(height - 1, -1, -1):
        if has_row_black(image, y):
            lower = y
            break
                
    return image.crop((left,upper,right + 1,lower + 1))

In [27]:
input_dir_2 = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_middle\\'
output_dir_2 = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_fonts\\'
default_size = 23

In [28]:
def centre_images(input_dir, output_dir):
    
    iteration = 0

    for filename in os.listdir(input_dir):
        image = Image.open(input_dir + filename)
        cropped_image = crop_image(image)
        image_background = Image.new(image_mode, (default_size, default_size), color=255)
        width, height = cropped_image.size
        new_width = default_size
        new_height = default_size

        left = 0
        upper = 0

        if width >= height:
            new_height = int(round(float(height) / width * default_size))
            upper = int((default_size - new_height) / 2.0)
        else:
            new_width = int(round(float(width) / height * default_size))
            left = int((default_size - new_width) / 2.0)

        resized_image = cropped_image.resize((new_width, new_height))
        image_background.paste(resized_image, (left, upper))        
        image_background.save(output_dir + filename, 'BMP')

In [29]:
for index in range(len(font_names)):
    if not os.path.exists(output_dir_2 + font_names[index]):
        prepare_dir(output_dir_2, font_names[index])
        centre_images(input_dir_2 + font_names[index] + '\\', output_dir_2 + font_names[index] + '\\')

Добавим гауссовский шум

In [31]:
import random
gauss_dir = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_fonts_gauss\\'

In [32]:
def gauss_images(input_dir, output_dir):
    for filename in os.listdir(input_dir):

IndentationError: expected an indented block (<ipython-input-32-88a46c81236f>, line 2)